Primero cargaremos el csv de los datos y luego analizaremos los atributos, limpiandolos y quedandonos con lo mas importante.

In [20]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler


In [3]:
# Ruta al archivo CSV
archivo_csv = 'Datasets/240310F1_Dataset.csv'
encoder = LabelEncoder()
# Leer el archivo CSV
df_F1 = pd.read_csv(archivo_csv)


df_F1.drop('DesgasteVuelta', axis=1, inplace=True)
df_F1.drop('Clima', axis=1, inplace=True)
df_F1.drop('TemperaturaPista', axis=1, inplace=True)

encoder = LabelEncoder()
df_F1['Piloto'] = encoder.fit_transform(df_F1['Piloto'])

fuel_discretizer = KBinsDiscretizer(n_bins=20, encode='ordinal', strategy='uniform')
Wheel_life_discretizer = KBinsDiscretizer(n_bins = 6, encode='ordinal', strategy='uniform')
Trust_Discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
#Water_Discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')

#df_F1['CargaCombustible'] = fuel_discretizer.fit_transform(df_F1[['CargaCombustible']])
#df_F1['VidaNeumatico'] = Wheel_life_discretizer.fit_transform(df_F1[['VidaNeumatico']])
df_F1['ConfianzaPilotoCoche'] = Trust_Discretizer.fit_transform(df_F1[['ConfianzaPilotoCoche']])
df_F1['ConfianzaPilotoPista'] = Trust_Discretizer.fit_transform(df_F1[['ConfianzaPilotoPista']])
#df_F1['CantidadAgua'] = Water_Discretizer.fit_transform(df_F1[['CantidadAgua']])

df_F1.dropna(inplace=True)

# Eliminar filas donde 'SafetyCar' es True
df_F1 = df_F1[df_F1['SafetyCar'] == False]
df_F1.drop('SafetyCar', axis=1, inplace=True)


# Eliminar filas donde haya trafico ya que no son representativas.
df_F1 = df_F1[df_F1['Trafico'] == False]
df_F1.drop('Trafico', axis=1, inplace=True)


df_F1 = pd.get_dummies(df_F1, columns = ['Circuito'] )
df_F1 = pd.get_dummies(df_F1, columns = ['Neumáticos'] )
df_F1 = pd.get_dummies(df_F1, columns = ['ModoPilotaje'] )
df_F1 = pd.get_dummies(df_F1, columns = ['GomaEnPista'] )
df_F1 = pd.get_dummies(df_F1, columns = ['AgarreAsfalto'] )



/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [4]:

# Selecting the 'CargaCombustible' column
carga_combustible = df_F1['CargaCombustible'].values.reshape(-1, 1)

min_max_scaler = MinMaxScaler()
carga_combustible_minmax = min_max_scaler.fit_transform(carga_combustible)

df_F1['CargaCombustible'] = carga_combustible_minmax


# Selecting the 'VidaNeumatico' column
Vida_neumatico = df_F1['VidaNeumatico'].values.reshape(-1, 1)

vida_neumatico_min_max = min_max_scaler.fit_transform(Vida_neumatico)

df_F1['VidaNeumatico'] = vida_neumatico_min_max


# Selecting the 'CantidadAgua' column
Cantidad_Agua = df_F1['CantidadAgua'].values.reshape(-1, 1)

Cantidad_Agua_min_max = min_max_scaler.fit_transform(Cantidad_Agua)

df_F1['CantidadAgua'] = Cantidad_Agua_min_max


X = df_F1.drop('TiempoVuelta', axis=1)
print(X)
Y = df_F1[['TiempoVuelta']]

      Entrenamientos  CargaCombustible  Sobrecalentamiento  VidaNeumatico  \
0               True          0.406827               False       0.955224   
1               True          0.320111               False       0.955224   
2               True          0.389299               False       0.895522   
3               True          0.301661               False       0.895522   
4               True          0.372694               False       0.835821   
...              ...               ...                 ...            ...   
1523           False          0.036900               False       0.119403   
1524           False          0.014760               False       0.208955   
1525           False          0.019373               False       0.059701   
1526           False          0.000000               False       0.149254   
1527           False          0.004613               False       0.000000   

      CantidadAgua  ConfianzaPilotoPista  ConfianzaPilotoCoche  Piloto  \
0

Ahora vamos a utilizar técnicas de mineria de datos para encontrar el mejor algoritmo que aproxime el mejor tiempo por vuelta dependiendo de los diferentes factores en los que se encuentra el coche.


Vamos a probar los siguientes Métodos: Regresion Lineal, SVM, Árboles de decisión, Random Forest

Modelo de regresion Lineal:

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression

modelo = LinearRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")



Mean Squared Error: 1.704370164685799
Error Medio: 0.5282552675189386


Ahora probaremos el método de vecinos mas cercanos.

In [6]:
from sklearn.neighbors import KNeighborsRegressor
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Crear el modelo KNN
knn = KNeighborsRegressor(n_neighbors=5) # Puedes ajustar el número de vecinos aquí
knn.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")


Mean Squared Error: 1.4066045636363644
Error Medio: 0.4035924242424236


Ahora probaremos los arboles de decision

In [7]:
from sklearn.tree import DecisionTreeRegressor

modelo = DecisionTreeRegressor(random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")


Mean Squared Error: 0.2875110530303026
Error Medio: 0.24715151515151443


Ahora probaremos los SVM

In [7]:
from sklearn.svm import SVR

modelo = SVR()
modelo.fit(X_train, y_train)

# Predecir el tiempo de vuelta en el conjunto de prueba
y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")


Mean Squared Error: 1.742731239160433
Error Medio: 0.4819402320446148


/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Ahora Probaremos el Random Forest

In [11]:
from sklearn.ensemble import RandomForestRegressor

modelo = RandomForestRegressor(n_estimators=5,  random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")


Mean Squared Error: 0.9680342492424239
Error Medio: 0.30999772727272656


Utilizamos ahora el Gradient Boosting, una tecnica de Boosting entrena sucesivamente diferentes modelos, centrandose en los fallos del modelo anterior.

In [9]:
from sklearn.ensemble import GradientBoostingRegressor

# Crear el modelo de Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=20, learning_rate=0.3, random_state=42)
gb.fit(X_train, y_train)

# Realizar predicciones y evaluar
y_pred = gb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")

Mean Squared Error: 0.8749150563466419
Error Medio: 0.4547218586758233


/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


A continuación vamos a ver el método de Stacking. Con el cual combinamos diferentes estimadores y el modelo se encarga de utilizar cada uno de los métodos dependiendo de cual estime mejor.

In [10]:
from sklearn.ensemble import StackingRegressor

# Definir los modelos base
estimators = [
    ('td', LinearRegression()),
    ('gb', GradientBoostingRegressor(n_estimators=20, learning_rate=0.3, random_state=42)),
]

# Crear el modelo de Stacking
stack_reg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stack_reg.fit(X_train, y_train)

# Realizar predicciones y evaluar
y_pred = stack_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")

/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error: 0.9450432654832265
Error Medio: 0.4096843664690793


Ahora vamos a probar el método de bagging

In [11]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# Crear el modelo de Bagging
bag_reg = BaggingRegressor(base_estimator=LinearRegression(), n_estimators=100, random_state=42)
bag_reg.fit(X_train, y_train)


# Realizar predicciones y evaluar
y_pred = bag_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

# Evaluar el modelo
mse = mean_squared_error(y_pred, y_test)
me = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Error Medio: {me}")

/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/javice/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Mean Squared Error: 1.4899468887603253
Error Medio: 0.4420734132405801


Ahora vamos a utilizar la validación cruzada para ver si obtenemos unos mejores resultados.

Con la regresion Lineal

In [9]:
import numpy as np
from sklearn.model_selection import KFold

# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")


Error Cuadrático Medio Promedio: 1.0653868874597154
Error Absoluto Medio Promedio: 0.47376715383818635


Ahora probaremos con KNN

In [13]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 1.752567991545454
Error Absoluto Medio Promedio: 0.48267954545454533


Ahora probaremos con los arboles de decisión

In [14]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")


Error Cuadrático Medio Promedio: 0.5434924318181817
Error Absoluto Medio Promedio: 0.26759242424242435


Ahora lo probamos con el SVM

In [15]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = SVR()
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 1.2237387609200145
Error Absoluto Medio Promedio: 0.46349018161147465


Con el random Forest

In [12]:
# Supongamos que df es tu DataFrame y que 'target' es la columna objetivo
df = df_F1 # Reemplaza con tu DataFrame
X = df.drop('TiempoVuelta', axis=1)  # Reemplaza 'target' con el nombre de tu columna objetivo
y = df['TiempoVuelta']  # De nuevo, reemplaza 'target' con el nombre de tu columna objetivo
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = RandomForestRegressor(n_estimators=200)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 0.5402377414984556
Error Absoluto Medio Promedio: 0.248193704545458


Ahora probaremos con el gradient Boosting

In [17]:

df = df_F1 
X = df.drop('TiempoVuelta', axis=1)  
y = df['TiempoVuelta']  
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.3, random_state=42)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 0.6463832167669709
Error Absoluto Medio Promedio: 0.29829603540318084


Ahora utilizaremos el método de Stacking

In [22]:

df = df_F1 
X = df.drop('TiempoVuelta', axis=1) 
y = df['TiempoVuelta']  
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    estimators = [
        ('rf', LinearRegression()),
        ('gb', GradientBoostingRegressor(n_estimators=100, learning_rate=0.3, random_state=42)),
    ]

    # Crear y entrenar el modelo
    model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

Error Cuadrático Medio Promedio: 0.7927169932692646
Error Absoluto Medio Promedio: 0.40932147097609944


Ahora lo probaremos con el bagging

In [13]:

df = df_F1 
X = df.drop('TiempoVuelta', axis=1) 
y = df['TiempoVuelta'] 
# Configuración de K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # shuffle y random_state son opcionales pero recomendados

mse_scores = []
mae_scores = []

# Bucle K-Fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear y entrenar el modelo
    model = BaggingRegressor(estimator=LinearRegression(), n_estimators=20, random_state=42)
    model.fit(X_train, y_train)

    # Hacer predicciones y evaluar
    y_pred = model.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))

# Calcular la media de los errores
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Error Cuadrático Medio Promedio: {mean_mse}")
print(f"Error Absoluto Medio Promedio: {mean_mae}")

NameError: name 'BaggingRegressor' is not defined

Ahora vamos a probar a ver si creando un modelo para cada circuito obtenemos mejores datos, ya que eso se ajustaría mejor a la realidad del problema.

Ahora vamos a definir la funcion que transformará los datos de entrada a los datos que necesita el modelo para funcionar correctamente:

In [24]:
def transform_data(Circuit, sesion, Neumaticos, ModoPilotaje, 
            CargaCombustible, GomaEnpista,
            Agarre, Sobrecalentamiento, VidaNeumatico, agua, confpilotopista,
            confpilotocoche, piloto, combustible):
    
    if piloto == "Sainz":
        piloto = 0
    else: piloto = 1


    if Circuit == 'Bahrein':
        Circuito_Barein = True
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Jeddah':
        Circuito_Barein = False
        Circuito_Jeddah = True
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Australia':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = True
        Circuito_Imola = False
        Circuito_Miami = False
    elif Circuit == 'Imola':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = True
        Circuito_Miami = False
    elif Circuit == 'Miami':
        Circuito_Barein = False
        Circuito_Jeddah = False
        Circuito_Australia = False
        Circuito_Imola = False
        Circuito_Miami = True
    

    if Neumaticos == 'Soft':
        Neumaticos_Soft = True
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Medium':
        Neumaticos_Soft = False
        Neumaticos_Medium = True
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Hard':
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = True
        Neumaticos_Inter = False
        Neumaticos_Rain = False
    elif Neumaticos == 'Inter':
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = True
        Neumaticos_Rain = False
    else:
        Neumaticos_Soft = False
        Neumaticos_Medium = False
        Neumaticos_Hard = False
        Neumaticos_Inter = False
        Neumaticos_Rain = True

    if ModoPilotaje == 'Push':
        ModoPilotaje_Push = True
        ModoPilotaje_Balanced = False
        ModoPilotaje_Save = False
    elif ModoPilotaje == 'Balanced':
        ModoPilotaje_Push = False
        ModoPilotaje_Balanced = True
        ModoPilotaje_Save = False
    elif ModoPilotaje == 'Save':
        ModoPilotaje_Push = False
        ModoPilotaje_Balanced = False
        ModoPilotaje_Save = True

    if GomaEnpista == 'VeryLow':
        GomaEnpista_VeryLow = True
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'Low':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = True
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'Average':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = True
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'High':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = True
        GomaEnpista_VeryHigh = False
    elif GomaEnpista == 'VeryHigh':
        GomaEnpista_VeryLow = False
        GomaEnpista_Low = False
        GomaEnpista_Average = False
        GomaEnpista_High = False
        GomaEnpista_VeryHigh = True

    if Agarre == 'Low':
        Agarre_Low = True
        Agarre_Average = False
        Agarre_High = False
        Agarre_VeryHigh = False
    elif Agarre == 'Average':
        Agarre_Low = False
        Agarre_Average = True
        Agarre_High = False
        Agarre_VeryHigh = False
    elif Agarre == 'High':
        Agarre_Low = False
        Agarre_Average = False
        Agarre_High = True
        Agarre_VeryHigh = False
    elif Agarre == 'VeryHigh':
        Agarre_Low = False
        Agarre_Average = False
        Agarre_High = False
        Agarre_VeryHigh = True


    new_df = pd.DataFrame({
        
        'Entrenamientos' : [sesion],
        'CargaCombustible' : [CargaCombustible],
        'Sobrecalentamiento' : [Sobrecalentamiento],
        'VidaNeumatico' : [VidaNeumatico],
        'CantidadAgua' : [agua],
        'ConfianzaPilotoPista' : [confpilotopista],
        'ConfianzaPilotoCoche' : [confpilotocoche],
        'Piloto' : [piloto],
        'CombustibleAlto' : [combustible],
        'Circuito_Australia' : [Circuito_Australia],
        'Circuito_Bahrein' : [Circuito_Barein],
        'Circuito_Imola' : [Circuito_Imola],
        'Circuito_Jeddah' : [Circuito_Jeddah],
        'Circuito_Miami' : [Circuito_Miami],
        'Neumáticos_Hard' : [Neumaticos_Hard],
        'Neumáticos_Inter' : [Neumaticos_Inter],
        'Neumáticos_Medium' : [Neumaticos_Medium],
        'Neumáticos_Rain' : [Neumaticos_Rain],
        'Neumáticos_Soft' : [Neumaticos_Soft],
        'ModoPilotaje_Balanced' : [ModoPilotaje_Balanced],
        'ModoPilotaje_Push' : [ModoPilotaje_Push],
        'ModoPilotaje_Save' : [ModoPilotaje_Save],
        'GomaEnPista_Average' : [GomaEnpista_Average],
        'GomaEnPista_High' : [GomaEnpista_High],
        'GomaEnPista_Low' : [GomaEnpista_Low],
        'GomaEnPista_VeryHigh' : [GomaEnpista_VeryHigh],
        'GomaEnPista_VeryLow' : [GomaEnpista_VeryLow],
        'AgarreAsfalto_Average' : [Agarre_Average],
        'AgarreAsfalto_High' : [Agarre_High],
        'AgarreAsfalto_Low' : [Agarre_Low],
        'AgarreAsfalto_VeryHigh' : [Agarre_VeryHigh]
    })

    return new_df